I have created some copies of this file, so feel free to edit and make changes

In [ ]:
!pip install torch torchvision torchaudio
!pip install easyocr
!pip install transformers
!pip install opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 22.2 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import logging
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import urllib.request
import time

import easyocr  # Deep learning-based OCR
from transformers import pipeline  # NER from Transformers
import re  # Regular expressions for pattern matching



In [ ]:
# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])  # Initialize reader for English language

# Initialize NER Pipeline using Transformers
ner = pipeline("ner", model="dslim/bert-base-NER")

# Setup logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Correct paths for the CSV files
train_csv_file_path = r'/content/train.csv'
test_csv_file_path = r'/content/test.csv'


# Load the Data
logging.info("Loading training and testing data from CSV files.")
train_df = pd.read_csv(train_csv_file_path)
test_df = pd.read_csv(test_csv_file_path)

# Function to create directory if it does not exist
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        logging.info(f"Created directory: {directory_path}")
    else:
        logging.info(f"Directory already exists: {directory_path}")

# Function to download images
def download_image(image_link, save_folder, retries=3, delay=3):
    # Ensure the save folder exists
    create_directory(save_folder)

    if not isinstance(image_link, str):
        logging.warning(f"Invalid image link: {image_link}")
        return None

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        logging.info(f"Image already exists at {image_save_path}, skipping download.")
        return image_save_path

    for attempt in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            logging.info(f"Downloaded image: {image_save_path}")
            return image_save_path
        except Exception as e:
            logging.warning(f"Failed to download {image_link} on attempt {attempt + 1}. Error: {e}")
            time.sleep(delay)

    logging.error(f"Failed to download image after {retries} attempts: {image_link}")
    return None

# Optimized concurrent image download
def download_images_concurrently(image_links, save_folder):
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(download_image, link, save_folder): link for link in image_links}
        for future in as_completed(futures):
            link = futures[future]
            try:
                future.result()
            except Exception as exc:
                logging.error(f"Image download failed for {link}: {exc}")

# Function to detect and extract text from images
def extract_text_from_image(image_path):
    results = reader.readtext(image_path)
    extracted_text = ' '.join([result[1] for result in results])  # Combine all detected text into a single string
    return extracted_text

# Regex patterns for extracting entity features
'''entity_patterns = {
    "item_weight": r'(\d+(\.\d+)?\s?(gram|kilogram|microgram|milligram|ounce|pound|ton))',
    "item_volume": r'(\d+(\.\d+)?\s?(centilitre|cubic foot|cubic inch|cup|decilitre|fluid ounce|gallon|imperial gallon|litre|microlitre|millilitre|pint|quart))',
    "height": r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard))',
    "width": r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard))',
    "dimension": r'(\d+(\.\d+)?\s?(mm|cm|m|in|ft|yd))',
    "voltage": r'(\d+(\.\d+)?\s?(volt|kilovolt|millivolt))',
    "wattage": r'(\d+(\.\d+)?\s?(watt|kilowatt))'
}'''
entity_patterns = {
    "item_weight": r'(\d+(\.\d+)?\s?(gram|kilogram|microgram|milligram|ounce|pound|ton|g|kg|mg|lb))',
    "item_volume": r'(\d+(\.\d+)?\s?(centilitre|cubic foot|cubic inch|cup|decilitre|fluid ounce|gallon|imperial gallon|litre|microlitre|millilitre|pint|quart|cl|ml|l|fl oz|gal))',
    "height": r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard|cm|ft|in|mm|m|yd))',
    "width": r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard|cm|ft|in|mm|m|yd))',
    "depth": r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard|cm|ft|in|mm|m|yd))',
    "dimension": r'(\d+(\.\d+)?\s?(mm|cm|m|in|ft|yd))',
    "voltage": r'(\d+(\.\d+)?\s?(volt|kilovolt|millivolt|V|kV|mV))',
    "wattage": r'(\d+(\.\d+)?\s?(watt|kilowatt|W|kW))',
    "maximum_weight_recommendation": r'(\d+(\.\d+)?\s?(gram|kilogram|microgram|milligram|ounce|pound|ton|g|kg|mg|lb))'
}

# Function to extract entities from text using NER
def extract_entities(text):
    entities = ner(text)
    entity_dict = {}
    for entity in entities:
        if entity['entity'] not in entity_dict:
            entity_dict[entity['entity']] = []
        entity_dict[entity['entity']].append(entity['word'])
    return entity_dict



Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Combine text extraction and entity classification
def extract_features_from_image(image_path):
    text = extract_text_from_image(image_path)
    extracted_entities = extract_entities(text)

    # Extract numerical values with units using regex
    entity_features = {}
    for entity_name, pattern in entity_patterns.items():
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            entity_features[entity_name] = [match[0] for match in matches]  # Extract the full matched pattern

    return text, entity_features

# Predict for Test Data
def generate_predictions(test_df, save_folder):
    # Limit processing to 1000 unique images
    image_links = test_df['image_link'].unique()[:limit]

    # Download the first 1000 images
    download_images_concurrently(image_links, save_folder)

    results = []
    for idx, row in test_df.head(limit).iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']
        index = row['index']

        image_filename = Path(image_url).name
        image_path = os.path.join(save_folder, image_filename)

        if not os.path.exists(image_path):
            logging.warning(f"Image not found: {image_path}")
            continue

        text, entity_features = extract_features_from_image(image_path)
        results.append({"index": index, "text": text, "extracted_entities": entity_features})

        logging.info(f"Processed index {index}: Extracted Text - {text}, Entities - {entity_features}")

    return pd.DataFrame(results)




In [ ]:
# Run the prediction generation
logging.info("Starting prediction generation for test data.")
save_folder = r'/content/Someresults'
create_directory(save_folder)  # Ensure save folder is created
predictions_df = generate_predictions(test_df, save_folder)
predictions_df.to_csv('output_advanced_features1.csv', index=False)
logging.info("Predictions saved to output_advanced_features.csv")

logging.info("Completed processing and saved the predictions.")

In [ ]:
import re
import pandas as pd
import logging
import os
import time
import urllib.request
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import pipeline
import easyocr
from PIL import Image

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])  # Initialize reader for English language

# Initialize NER Pipeline using Transformers
ner = pipeline("ner", model="dslim/bert-base-NER")

# Setup logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Correct paths for the CSV files
train_csv_file_path = r'/content/train.csv'
test_csv_file_path = r'/content/test.csv'

# Load the Data
logging.info("Loading training and testing data from CSV files.")
train_df = pd.read_csv(train_csv_file_path)
test_df = pd.read_csv(test_csv_file_path)

# Function to create directory if it does not exist
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        logging.info(f"Created directory: {directory_path}")
    else:
        logging.info(f"Directory already exists: {directory_path}")

# Function to download images
def download_image(image_link, save_folder, retries=3, delay=3):
    # Ensure the save folder exists
    create_directory(save_folder)

    if not isinstance(image_link, str):
        logging.warning(f"Invalid image link: {image_link}")
        return None

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        logging.info(f"Image already exists at {image_save_path}, skipping download.")
        return image_save_path

    for attempt in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            logging.info(f"Downloaded image: {image_save_path}")
            return image_save_path
        except Exception as e:
            logging.warning(f"Failed to download {image_link} on attempt {attempt + 1}. Error: {e}")
            time.sleep(delay)

    logging.error(f"Failed to download image after {retries} attempts: {image_link}")
    return None

# Optimized concurrent image download
def download_images_concurrently(image_links, save_folder):
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(download_image, link, save_folder): link for link in image_links}
        for future in as_completed(futures):
            link = futures[future]
            try:
                future.result()
            except Exception as exc:
                logging.error(f"Image download failed for {link}: {exc}")

# Function to detect and extract text from images
def extract_text_from_image(image_path):
    results = reader.readtext(image_path)
    extracted_text = ' '.join([result[1] for result in results])  # Combine all detected text into a single string
    return extracted_text

# Regex patterns for extracting entity features
entity_patterns = {
    "item_weight": re.compile(r'(\d+(\.\d+)?\s?(gram|kilogram|microgram|milligram|ounce|pound|ton))', re.IGNORECASE),
    "item_volume": re.compile(r'(\d+(\.\d+)?\s?(centilitre|cubic foot|cubic inch|cup|decilitre|fluid ounce|gallon|imperial gallon|litre|microlitre|millilitre|pint|quart))', re.IGNORECASE),
    "height": re.compile(r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard))', re.IGNORECASE),
    "width": re.compile(r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard))', re.IGNORECASE),
    "dimension": re.compile(r'(\d+(\.\d+)?\s?(mm|cm|m|in|ft|yd))', re.IGNORECASE),
    "voltage": re.compile(r'(\d+(\.\d+)?\s?(volt|kilovolt|millivolt))', re.IGNORECASE),
    "wattage": re.compile(r'(\d+(\.\d+)?\s?(watt|kilowatt))', re.IGNORECASE)
}

# Entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Add all units from entity_unit_map to regex patterns
for entity, units in entity_unit_map.items():
    pattern = r'(\d+(\.\d+)?\s?(' + '|'.join(units) + r'))'
    entity_patterns[entity] = re.compile(pattern, re.IGNORECASE)

# Function to extract entities from text using NER and regex
def extract_entities_advanced(text):
    # Use NLP model to extract entities
    doc = nlp(text)
    entity_features = defaultdict(list)

    # Extract entities based on NER
    for ent in doc.ents:
        if ent.label_ in {'QUANTITY', 'CARDINAL'}:
            for entity_name, units in entity_patterns.items():
                if any(unit in ent.text.lower() for unit in entity_patterns[entity_name].pattern.split('|')):
                    entity_features[entity_name].append(ent.text)

    # Also use regex to find more complex patterns
    for entity_name, pattern in entity_patterns.items():
        matches = pattern.findall(text)
        if matches:
            entity_features[entity_name].extend([match[0] for match in matches])

    # Remove duplicates and empty entries
    entity_features = {k: list(set(v)) for k, v in entity_features.items() if v}

    return entity_features

# Function to extract entities from text using NER
def extract_entities(text):
    entities = ner(text)
    entity_dict = {}
    for entity in entities:
        if entity['entity'] not in entity_dict:
            entity_dict[entity['entity']] = []
        entity_dict[entity['entity']].append(entity['word'])
    return entity_dict

# Updated function to extract text and entities
def extract_features_from_image(image_path):
    text = extract_text_from_image(image_path)
    cleaned_text = clean_text(text)
    entity_features = extract_entities_advanced(cleaned_text)
    return cleaned_text, entity_features

# Sample debug to see cleaned text and extracted entities
def debug_sample_extraction(image_path):
    cleaned_text, entity_features = extract_features_from_image(image_path)
    logging.info(f"Cleaned Text: {cleaned_text}")
    logging.info(f"Extracted Entities: {entity_features}")

# Test with a sample image to check the extraction process




/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [ ]:
'''sample_image_path = '/content/Someresults/110EibNyclL.jpg'  # Replace with actual path
debug_sample_extraction(sample_image_path)
'''

"sample_image_path = '/content/Someresults/110EibNyclL.jpg'  # Replace with actual path\ndebug_sample_extraction(sample_image_path)\n"

In [ ]:
def debug_sample_extraction(image_path):
    cleaned_text, entity_features = extract_features_from_image(image_path)
    print(f"Cleaned Text: {cleaned_text}")
    print(f"Extracted Entities: {entity_features}")


In [ ]:
!pip install tqdm

In [ ]:
import os
import pandas as pd
import logging
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import urllib.request
import time

import easyocr  # Deep learning-based OCR
from transformers import pipeline  # NER from Transformers
import re  # Regular expressions for pattern matching
from tqdm import tqdm  # For progress bar
from IPython.display import display, clear_output

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])  # Initialize reader for English language

# Initialize NER Pipeline using Transformers
ner = pipeline("ner", model="dslim/bert-base-NER")

# Setup logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Correct paths for the CSV files
train_csv_file_path = r'/content/train.csv'
test_csv_file_path = r'/content/test.csv'

# Load the Data
logging.info("Loading training and testing data from CSV files.")
train_df = pd.read_csv(train_csv_file_path)
test_df = pd.read_csv(test_csv_file_path)

# Function to create directory if it does not exist
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        logging.info(f"Created directory: {directory_path}")
    else:
        logging.info(f"Directory already exists: {directory_path}")

# Function to download images
def download_image(image_link, save_folder, retries=3, delay=3):
    # Ensure the save folder exists
    create_directory(save_folder)

    if not isinstance(image_link, str):
        logging.warning(f"Invalid image link: {image_link}")
        return None

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        logging.info(f"Image already exists at {image_save_path}, skipping download.")
        return image_save_path

    for attempt in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            logging.info(f"Downloaded image: {image_save_path}")
            return image_save_path
        except Exception as e:
            logging.warning(f"Failed to download {image_link} on attempt {attempt + 1}. Error: {e}")
            time.sleep(delay)

    logging.error(f"Failed to download image after {retries} attempts: {image_link}")
    return None

# Optimized concurrent image download
def download_images_concurrently(image_links, save_folder):
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(download_image, link, save_folder): link for link in image_links}
        for future in as_completed(futures):
            link = futures[future]
            try:
                future.result()
            except Exception as exc:
                logging.error(f"Image download failed for {link}: {exc}")

# Function to detect and extract text from images
def extract_text_from_image(image_path):
    results = reader.readtext(image_path)
    extracted_text = ' '.join([result[1] for result in results])  # Combine all detected text into a single string
    return extracted_text

# Regex patterns for extracting entity features
entity_patterns = {
    "item_weight": r'(\d+(\.\d+)?\s?(gram|kilogram|microgram|milligram|ounce|pound|ton|g|kg|mg|lb))',
    "item_volume": r'(\d+(\.\d+)?\s?(centilitre|cubic foot|cubic inch|cup|decilitre|fluid ounce|gallon|imperial gallon|litre|microlitre|millilitre|pint|quart|cl|ml|l|fl oz|gal))',
    "height": r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard|cm|ft|in|mm|m|yd))',
    "width": r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard|cm|ft|in|mm|m|yd))',
    "depth": r'(\d+(\.\d+)?\s?(centimetre|foot|inch|metre|millimetre|yard|cm|ft|in|mm|m|yd))',
    "dimension": r'(\d+(\.\d+)?\s?(mm|cm|m|in|ft|yd))',
    "voltage": r'(\d+(\.\d+)?\s?(volt|kilovolt|millivolt|V|kV|mV))',
    "wattage": r'(\d+(\.\d+)?\s?(watt|kilowatt|W|kW))',
    "maximum_weight_recommendation": r'(\d+(\.\d+)?\s?(gram|kilogram|microgram|milligram|ounce|pound|ton|g|kg|mg|lb))'
}

# Function to extract entities from text using NER
def extract_entities(text):
    entities = ner(text)
    entity_dict = {}
    for entity in entities:
        if entity['entity'] not in entity_dict:
            entity_dict[entity['entity']] = []
        entity_dict[entity['entity']].append(entity['word'])
    return entity_dict

# Combine text extraction and entity classification
def extract_features_from_image(image_path):
    text = extract_text_from_image(image_path)
    extracted_entities = extract_entities(text)

    # Extract numerical values with units using regex
    entity_features = {}
    for entity_name, pattern in entity_patterns.items():
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            entity_features[entity_name] = [match[0] for match in matches]  # Extract the full matched pattern

    return text, entity_features

# Predict for Test Data
def generate_predictions(test_df, save_folder):
    # Limit processing to 1000 unique images
    image_links = test_df['image_link'].unique()

    # Download the images with progress
    logging.info("Downloading images...")
    download_images_concurrently(image_links, save_folder)

    results = []
    progress_bar = tqdm(total=len(test_df), desc="Processing Images")

    for idx, row in test_df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']
        index = row['index']

        image_filename = Path(image_url).name
        image_path = os.path.join(save_folder, image_filename)

        if not os.path.exists(image_path):
            logging.warning(f"Image not found: {image_path}")
            continue

        text, entity_features = extract_features_from_image(image_path)
        results.append({"index": index, "text": text, "extracted_entities": entity_features})

        # Print current progress in Colab
        clear_output(wait=True)
        display(f"Processed index {index}: Extracted Text - {text}, Entities - {entity_features}")

        # Update progress bar
        progress_bar.update(1)

    progress_bar.close()
    return pd.DataFrame(results)

# Run the prediction generation
logging.info("Starting prediction generation for test data.")
save_folder = r'/content/Someresults'
create_directory(save_folder)  # Ensure save folder is created
predictions_df = generate_predictions(test_df, save_folder)
predictions_df.to_csv('output_advanced_features1.csv', index=False)
logging.info("Predictions saved to output_advanced_features1.csv")

logging.info("Completed processing and saved the predictions.")


"Processed index 16684: Extracted Text - 30cm/11.81in 40cm/15in 24pcs of twist pins, Entities - {'height': ['30cm', '11.81in', '40cm', '15in'], 'width': ['30cm', '11.81in', '40cm', '15in'], 'depth': ['30cm', '11.81in', '40cm', '15in'], 'dimension': ['30cm', '11.81in', '40cm', '15in']}"

Processing Images:  13%|█▎        | 16668/131187 [1:59:46<15:01:04,  2.12it/s]